I have prepared this data to model with multivariable regression according to this specification:

$$ Property crime = \alpha + log(Population) + fraction(larceny) + fraction(Murder) + fraction(car theft)$$

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf

%matplotlib inline

In [2]:
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls', header=4)
drop_df = df.drop('Rape\n(revised\ndefinition)1',axis=1) #removing column because only NaN values
crime_df = drop_df[:-3] #Last three rows are not data

In [3]:
property_crime = pd.DataFrame()
n = 10000 #per 1000 people

#features
property_crime['log_population'] = np.log(crime_df['Population'])
property_crime['frac_assault'] = (crime_df['Aggravated\nassault']/crime_df['Population'])*n
property_crime['frac_robbery'] = (crime_df['Robbery']/crime_df['Population'])*n
property_crime['frac_murder'] = (crime_df['Murder and\nnonnegligent\nmanslaughter']/crime_df['Population'])*n

#dependent variable
property_crime['log_property_crime'] = crime_df['Property\ncrime']

drop_index = np.concatenate((np.where(property_crime['log_population'] == property_crime['log_population'].max())[0], 
                            np.where(property_crime['log_property_crime'] == 0)[0],
                            property_crime['frac_assault'].nlargest(21).index,
                            property_crime['frac_robbery'].nlargest(20).index,
                            property_crime['frac_murder'].nlargest(5).index))
property_crime = property_crime.drop(drop_index, axis=0)

property_crime['log_property_crime'] = np.log(property_crime['log_property_crime'])

drop_index_new = np.concatenate((property_crime['log_property_crime'].nlargest(2).index, 
                                 property_crime['log_property_crime'].nsmallest(1).index))
property_crime = property_crime.drop(drop_index_new, axis=0)

property_crime.head()

,log_population,frac_assault,frac_robbery,frac_murder,log_property_crime
0,7.528869,0.000000,0.000000,0.0,2.484907
1,7.854381,11.641444,0.000000,0.0,3.178054
2,7.953670,10.541110,0.000000,0.0,2.772589
4,8.762177,25.046963,6.261741,0.0,5.407172
5,8.316056,4.891171,7.336757,0.0,3.828641


Here I chose to take the log values of the property crime and the population to normalize the distributions. Then I took the features for larceny, robbery, and murder and divided each one by the population so that the features is in fraction of the feature per population.

In [5]:
display(property_crime.head())

# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = property_crime['log_property_crime'].values.reshape(-1, 1)
X = property_crime[['log_population','frac_assault','frac_robbery','frac_murder']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

,log_population,frac_assault,frac_robbery,frac_murder,log_property_crime
0,7.528869,0.000000,0.000000,0.0,2.484907
1,7.854381,11.641444,0.000000,0.0,3.178054
2,7.953670,10.541110,0.000000,0.0,2.772589
4,8.762177,25.046963,6.261741,0.0,5.407172
5,8.316056,4.891171,7.336757,0.0,3.828641



Coefficients: 
 [[ 1.03447694  0.03243141  0.08676027 -0.07776877]]

Intercept: 
 [-5.00879398]

R-squared:
0.7961967978697575


Here, our model where the outcome Property Crime is predicted by the features population, larceny, robbery, and murder (percent of population) explains $79.6%$ of the variance in property crime. 

The intercept is negative and the highest increase in property crime comes from population (1.03)). 

In [ ]:
# Extract predicted values.
predicted = regr.predict(X).ravel()
actual = (property_crime['log_property_crime'])

# Calculate the error, also called the residual.
residual = actual - predicted

# This looks a bit concerning.
plt.hist(residual)
plt.title('Residual counts')
plt.xlabel('Residual')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.scatter(predicted, residual)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
plt.ylim(-5,5)
plt.xlim(0,10)
plt.show()

In [ ]:
correlation_matrix = X.corr()
display(correlation_matrix)

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(property_crime.corr())

plt.show()